# Part 1 - Extracting and Saving Data from Yelp API

## Obective

- For this CodeAlong, we will be working with the Yelp API. 
- You will use the the Yelp API to search your home town for a cuisine type of your choice.
- Next class, we will then use Plotly Express to create a map with the Mapbox API to visualize the results.
    
    

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- Efficient API Calls Lesson Link: https://login.codingdojo.com/m/376/12529/88078

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

## 1. Registering for Required APIs


- Yelp: https://www.yelp.com/developers/documentation/v3/get_started


> Check the official API documentation to know what arguments we can search for: https://www.yelp.com/developers/documentation/v3/business_search

### Load Credentials and Create Yelp API Object

In [2]:
# Load API Credentials
with open('/Users/purvikansara/.secret/yelp_api.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['Client-ID', 'API Key'])

In [3]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(login['API Key'],timeout_s=5.0)

### Define Search Terms and File Paths

In [4]:
# set our API call parameters and filename before the first call
location  = 'Seattle, WA 98122'
term = 'coffee'

In [5]:
location.split(',')[0]

'Seattle'

In [6]:
## Specify fodler for saving data

FOLDER = 'Data/'
os.makedirs(FOLDER,exist_ok=True)
# Specifying JSON_FILE filename (can include a folder)
#JSON_FILE = FOLDER+'Seattle pizza.json'


JSON_FILE =FOLDER+f"{location.split(',')[0]}-{term}.json"

In [7]:
JSON_FILE

'Data/Seattle-coffee.json'

### Make the first API call to get the first page of data

- We will use this first result to check:
    - how many total results there are?
    - Where is the actual data we want to save?
    - how many results do we get at a time?


In [8]:
# use our yelp_api variable's search_query method to perform our API call

results =yelp.search_query(term=term,location=location)

type(results)

dict

In [9]:
results

{'businesses': [{'id': 'yCLUXtS6a1xMicPSsmdTfA',
   'alias': 'squirrel-chops-seattle',
   'name': 'Squirrel Chops',
   'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/0fA2ebtcBXd5Qi6UE6iUTA/o.jpg',
   'is_closed': False,
   'url': 'https://www.yelp.com/biz/squirrel-chops-seattle?adjust_creative=r8Qlusnw5U_OXTmzDH90Ug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=r8Qlusnw5U_OXTmzDH90Ug',
   'review_count': 91,
   'categories': [{'alias': 'coffee', 'title': 'Coffee & Tea'}],
   'rating': 4.5,
   'coordinates': {'latitude': 47.6128151, 'longitude': -122.3034697},
   'transactions': [],
   'price': '$',
   'location': {'address1': '2201 E Union St',
    'address2': '',
    'address3': None,
    'city': 'Seattle',
    'zip_code': '98122',
    'country': 'US',
    'state': 'WA',
    'display_address': ['2201 E Union St', 'Seattle, WA 98122']},
   'phone': '+12064857017',
   'display_phone': '(206) 485-7017',
   'distance': 572.7238765862401},
  {'id': 'mUtrinuyz

In [10]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [11]:
results['businesses'][0]

{'id': 'yCLUXtS6a1xMicPSsmdTfA',
 'alias': 'squirrel-chops-seattle',
 'name': 'Squirrel Chops',
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/0fA2ebtcBXd5Qi6UE6iUTA/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/squirrel-chops-seattle?adjust_creative=r8Qlusnw5U_OXTmzDH90Ug&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=r8Qlusnw5U_OXTmzDH90Ug',
 'review_count': 91,
 'categories': [{'alias': 'coffee', 'title': 'Coffee & Tea'}],
 'rating': 4.5,
 'coordinates': {'latitude': 47.6128151, 'longitude': -122.3034697},
 'transactions': [],
 'price': '$',
 'location': {'address1': '2201 E Union St',
  'address2': '',
  'address3': None,
  'city': 'Seattle',
  'zip_code': '98122',
  'country': 'US',
  'state': 'WA',
  'display_address': ['2201 E Union St', 'Seattle, WA 98122']},
 'phone': '+12064857017',
 'display_phone': '(206) 485-7017',
 'distance': 572.7238765862401}

In [12]:
pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,yCLUXtS6a1xMicPSsmdTfA,squirrel-chops-seattle,Squirrel Chops,https://s3-media2.fl.yelpcdn.com/bphoto/0fA2eb...,False,https://www.yelp.com/biz/squirrel-chops-seattl...,91,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.5,"{'latitude': 47.6128151, 'longitude': -122.303...",[],$,"{'address1': '2201 E Union St', 'address2': ''...",+12064857017,(206) 485-7017,572.723877
1,mUtrinuyznLic5wYc09LeA,arosa-cafe-seattle-6,Arosa Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/Sz1XcR...,False,https://www.yelp.com/biz/arosa-cafe-seattle-6?...,109,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 47.62622241003878, 'longitude': -...","[pickup, delivery]",$,"{'address1': '3121 E Madison St', 'address2': ...",+12064050605,(206) 405-0605,1487.954980
2,Q8Nlx_FeZ4LW0kdbnHoyrw,qed-coffee-seattle,QED Coffee,https://s3-media3.fl.yelpcdn.com/bphoto/HwagNI...,False,https://www.yelp.com/biz/qed-coffee-seattle?ad...,105,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.5,"{'latitude': 47.5896816513796, 'longitude': -1...",[delivery],$$,"{'address1': '1418 31st Ave S', 'address2': ''...",+12062357678,(206) 235-7678,2615.675293
3,4o7FQKh8GXSOjoJa1OXfsw,overcast-coffee-company-seattle,Overcast Coffee Company,https://s3-media2.fl.yelpcdn.com/bphoto/xz55iC...,False,https://www.yelp.com/biz/overcast-coffee-compa...,41,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",5.0,"{'latitude': 47.62092, 'longitude': -122.31256}","[pickup, delivery]",$,"{'address1': '312 15th Ave E', 'address2': '',...",+12063222220,(206) 322-2220,1529.086775
4,2ZrovOWUNfzpSJlx-7DYCw,hello-em-viet-coffee-and-roastery-seattle-2,Hello Em Viet Coffee & Roastery,https://s3-media2.fl.yelpcdn.com/bphoto/Wua0EA...,False,https://www.yelp.com/biz/hello-em-viet-coffee-...,252,"[{'alias': 'coffeeroasteries', 'title': 'Coffe...",4.5,"{'latitude': 47.59732, 'longitude': -122.31544}",[],$,"{'address1': '1227 S Weller St', 'address2': '...",,,2296.777516
5,FVzl8rDPiTWEtrNEuCu-Xg,storyville-coffee-company-seattle-9,Storyville Coffee Company,https://s3-media3.fl.yelpcdn.com/bphoto/TecGMk...,False,https://www.yelp.com/biz/storyville-coffee-com...,2080,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 47.60895949363687, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '94 Pike St', 'address2': 'Ste 34...",+12067805777,(206) 780-5777,3385.734739
6,NODuSm6R6hfvoONZbrVuJQ,url-coffee-seattle,URL Coffee,https://s3-media4.fl.yelpcdn.com/bphoto/Jgm-IR...,False,https://www.yelp.com/biz/url-coffee-seattle?ad...,36,"[{'alias': 'cafes', 'title': 'Cafes'}]",5.0,"{'latitude': 47.60672503133874, 'longitude': -...",[delivery],NaN,"{'address1': '524 Broadway', 'address2': None,...",,,1992.130352
7,9TnUll8eOc5FREIQqrcrFg,verite-coffee-seattle,Verite Coffee,https://s3-media4.fl.yelpcdn.com/bphoto/SaSFeE...,False,https://www.yelp.com/biz/verite-coffee-seattle...,38,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.0,"{'latitude': 47.61175, 'longitude': -122.28953}",[],$,"{'address1': '1101 34th Ave', 'address2': '', ...",+12067094497,(206) 709-4497,483.915129
8,ebglfaWBLj-z7yqpG3BcsQ,golden-wheat-bakery-seattle,Golden Wheat Bakery,https://s3-media1.fl.yelpcdn.com/bphoto/YasEcz...,False,https://www.yelp.com/biz/golden-wheat-bakery-s...,129,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.5,"{'latitude': 47.608213, 'longitude': -122.294437}",[delivery],$,"{'address1': '2908 E Cherry St', 'address2': '...",+12063255055,(206) 325-5055,550.890653
9,cpfool0I6kNLpIe0Su6fqw,avole-ethiopian-coffee-seattle,Avole Ethiopian Coffee,https://s3-media3.fl.yelpcdn.com/bphoto/Wg18Mz...,False,https://www.yelp.com/biz/avole-ethiopian-coffe...,10,"[{'alias': 'juicebars', 'title': 'Juice Bars &...",5.0,"{'latitude': 47.61309227798837, 'longitude': -...",[],NaN,"{'address1': '2330 E Union St', 'address2': ''...",,,461.839660


In [13]:
results['total']


1400

In [14]:
results['region']

{'center': {'longitude': -122.29568481445312, 'latitude': 47.61309534151991}}

In [15]:
results_per_page = len(results['businesses'])

In [16]:
results_per_page

20

In [17]:
## How many results total?
# Use math.ceil to round up for the total number of pages of results.
import math
n_pages= math.ceil(results['total']/results_per_page)
n_pages

70

- Where is the actual data we want to save?

In [18]:
# lets save the data

# Check if Json file exists and create it if it does not

if os.path.isfile(JSON_FILE)==False:
    # file does not exist so print message and create empty file
    print(" The file does not exist. Creating empty file.")
    with open(JSON_FILE,'w') as f:
        json.dump(results['businesses'],f)
# if file exists
else:
    print("File already exists")

File already exists


In [19]:
for i in tqdm_notebook( range(1,n_pages+1)):
    ## The block of code we want to TRY to run
        
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE) as f:
            prev_results = json.load(f)
        
        ## save number of results for to use as offset
        n_results = len(prev_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp.search_query(term = term,location=location,offset=5)#n_results)

        ## append new results and save to file
        
        prev_results.extend(results['businesses'])
        with open(JSON_FILE,'w') as f:
            json.dump(prev_results,f)
            
    ## What to do if we get an error/exception.
        


  0%|          | 0/70 [00:00<?, ?it/s]

In [20]:
df = pd.read_json(JSON_FILE)
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,yCLUXtS6a1xMicPSsmdTfA,squirrel-chops-seattle,Squirrel Chops,https://s3-media2.fl.yelpcdn.com/bphoto/0fA2eb...,False,https://www.yelp.com/biz/squirrel-chops-seattl...,91,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.5,"{'latitude': 47.6128151, 'longitude': -122.303...",[],$,"{'address1': '2201 E Union St', 'address2': ''...",+12064857017,(206) 485-7017,572.723877
1,mUtrinuyznLic5wYc09LeA,arosa-cafe-seattle-6,Arosa Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/Sz1XcR...,False,https://www.yelp.com/biz/arosa-cafe-seattle-6?...,109,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 47.62622241003878, 'longitude': -...","[pickup, delivery]",$,"{'address1': '3121 E Madison St', 'address2': ...",+12064050605,(206) 405-0605,1487.954980
2,Q8Nlx_FeZ4LW0kdbnHoyrw,qed-coffee-seattle,QED Coffee,https://s3-media3.fl.yelpcdn.com/bphoto/HwagNI...,False,https://www.yelp.com/biz/qed-coffee-seattle?ad...,105,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.5,"{'latitude': 47.5896816513796, 'longitude': -1...",[delivery],$$,"{'address1': '1418 31st Ave S', 'address2': ''...",+12062357678,(206) 235-7678,2615.675293
3,4o7FQKh8GXSOjoJa1OXfsw,overcast-coffee-company-seattle,Overcast Coffee Company,https://s3-media2.fl.yelpcdn.com/bphoto/xz55iC...,False,https://www.yelp.com/biz/overcast-coffee-compa...,41,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",5.0,"{'latitude': 47.62092, 'longitude': -122.31256}","[pickup, delivery]",$,"{'address1': '312 15th Ave E', 'address2': '',...",+12063222220,(206) 322-2220,1529.086775
4,2ZrovOWUNfzpSJlx-7DYCw,hello-em-viet-coffee-and-roastery-seattle-2,Hello Em Viet Coffee & Roastery,https://s3-media2.fl.yelpcdn.com/bphoto/Wua0EA...,False,https://www.yelp.com/biz/hello-em-viet-coffee-...,252,"[{'alias': 'coffeeroasteries', 'title': 'Coffe...",4.5,"{'latitude': 47.59732, 'longitude': -122.31544}",[],$,"{'address1': '1227 S Weller St', 'address2': '...",,,2296.777516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2395,h1dhP2ZRiMGE2RdpUtputg,coffeeholic-house-seattle,Coffeeholic House,https://s3-media2.fl.yelpcdn.com/bphoto/dVvN0G...,False,https://www.yelp.com/biz/coffeeholic-house-sea...,589,"[{'alias': 'coffee', 'title': 'Coffee & Tea'}]",4.5,"{'latitude': 47.55723, 'longitude': -122.28596}",[delivery],$$,"{'address1': '3700 S Hudson St', 'address2': N...",+12067223327,(206) 722-3327,6248.940850
2396,AEdVTLYjw6rnmEkTCvqAPQ,katys-corner-cafe-seattle,Katy's Corner Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/CmXPP6...,False,https://www.yelp.com/biz/katys-corner-cafe-sea...,188,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.5,"{'latitude': 47.6130549, 'longitude': -122.306...","[pickup, delivery]",$,"{'address1': '2000 E Union St', 'address2': No...",+12063290121,(206) 329-0121,773.714431
2397,KGmwHp-94JPDpKCv0ParsA,madison-park-bakery-seattle,Madison Park Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/vs3R9a...,False,https://www.yelp.com/biz/madison-park-bakery-s...,88,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.0,"{'latitude': 47.63612, 'longitude': -122.27803}",[delivery],$$,"{'address1': '4214 E Madison St', 'address2': ...",+12063223238,(206) 322-3238,2876.739332
2398,MAlntNa6HiQ8k2xrB2vKRA,madison-kitchen-seattle,Madison Kitchen,https://s3-media2.fl.yelpcdn.com/bphoto/s6fbQd...,False,https://www.yelp.com/biz/madison-kitchen-seatt...,134,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",4.0,"{'latitude': 47.63548, 'longitude': -122.27895}",[delivery],$$,"{'address1': '4122 E Madison St', 'address2': ...",+12065574640,(206) 557-4640,2786.221642


In [21]:
# check for any duplicates
df.duplicated(subset='id').sum()

1407

In [22]:
# remove duplicates if any
df.drop_duplicates(subset='id',inplace=True)

In [23]:
df.duplicated(subset='id').sum()

0

In [24]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Seattle-coffee.csv.gz'

In [25]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file,compression='gzip',index=False)

## Bonus: compare filesize with os module's `os.path.getsize`

In [26]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 2,302,456 Bytes
CSV.GZ FILE: 137,605 Bytes
the csv.gz is 16.732357109116673 times smaller!


## Next Class: Processing the Results and Mapping 